In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV 

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
import time


In [2]:
df = pd.read_csv('../Data/final_inflation_adj_data.csv')

In [3]:
df.head()


,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime,month,rating_imputed,budget_imputed,inf_adjmnt_value,budget_adj,gross_adj,profit,profit_adj
0,The Shining,4.00,Drama,1980,8.40,"927,000.00",Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,"19,000,000.00","46,998,772.00",Warner Bros.,146.00,June,0,0,3.15,"59,842,877.87","148,028,514.36","27,998,772.00","88,185,636.49"
1,The Blue Lagoon,4.00,Adventure,1980,5.80,"65,000.00",Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,"4,500,000.00","58,853,106.00",Columbia Pictures,104.00,July,0,0,3.15,"14,173,313.18","185,365,222.88","54,353,106.00","171,191,909.70"
2,Star Wars: Episode V - The Empire Strikes Back,2.00,Action,1980,8.70,"1,200,000.00",Irvin Kershner,Leigh Brackett,Mark Hamill,United States,"18,000,000.00","538,375,067.00",Lucasfilm,124.00,June,0,0,3.15,"56,693,252.72","1,695,679,651.77","520,375,067.00","1,638,986,399.05"
3,Airplane!,2.00,Comedy,1980,7.70,"221,000.00",Jim Abrahams,Jim Abrahams,Robert Hays,United States,"3,500,000.00","83,453,539.00",Paramount Pictures,88.00,July,0,0,3.15,"11,023,688.03","262,847,365.39","79,953,539.00","251,823,677.36"
4,Caddyshack,4.00,Comedy,1980,7.30,"108,000.00",Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,"6,000,000.00","39,846,344.00",Orion Pictures,98.00,July,0,0,3.15,"18,897,750.91","125,501,047.24","33,846,344.00","106,603,296.34"


In [4]:
#separate numerical and categorical features
categorical_cols =['genre', 'month','country', 'company'] ##'country', 'company',
numerical_cols = ['runtime','rating','budget_adj']##'budget_adj',

In [5]:
ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = True)

In [6]:
#preprocessing steps
ss = StandardScaler()
ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = False)
ctx = ColumnTransformer(
                            transformers = [
                                 ('one_hot', ohe, categorical_cols ),
                                 ('sc',  ss,   numerical_cols)
                                ] 
                                   , verbose_feature_names_out=False
                                    , remainder='passthrough'
                        
                            ) 


In [7]:
X=df[categorical_cols + numerical_cols]
y= df['profit_adj']

In [8]:
X = ctx.fit_transform(X)

In [9]:
X_enc =pd.DataFrame(X, columns=ctx.get_feature_names_out())

In [10]:
X_enc.shape

(7475, 2360)

In [11]:
#split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_enc,y,test_size = 0.2, random_state =42)
X_train.shape, X_test.shape

((5980, 2360), (1495, 2360))

#### Perform GridSearch for different models

In [12]:
pipe_lr = Pipeline([('lr', LinearRegression())]
                   )
params = {}
gs_lr = GridSearchCV(pipe_lr, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_lr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('lr', LinearRegression())]),
             n_jobs=-1, param_grid={}, scoring='r2')

In [13]:
# Evaluate the  model 
y_pred = gs_lr.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)


print(f'Train rsquare: {gs_lr.score(X_train, y_train)}')
print(f'Test rsquare: {gs_lr.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.4755680360262601
Test rsquare: -2.8477296486926158e+23
RMSE: 9.446043331636961e+19


In [14]:
pipe_rd = Pipeline([('rd', Ridge())]
                   )
params = {'rd__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

gs_rd = GridSearchCV(pipe_rd, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_rd.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('rd', Ridge())]), n_jobs=-1,
             param_grid={'rd__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='r2')

In [16]:
# Evaluate the  model on the test set
y_pred = gs_rd.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f'Train rsquare: {gs_rd.score(X_train, y_train)}')
print(f'Test rsquare: {gs_rd.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.3891094002631844
Test rsquare: 0.37944210280336443
RMSE: 139441391.64616463


In [17]:
pipe_ls = Pipeline([('ls', Lasso())]
                   )
params = {'ls__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

gs_ls = GridSearchCV(pipe_ls, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_ls.fit(X_train, y_train)

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.126e+19, tolerance: 1.607e+16
  model = cd_fast.enet_coordinate_descent(
/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.128e+19, tolerance: 1.607e+16
  model = cd_fast.enet_coordinate_descent(
/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.099e+18, tolerance: 1.522e+16
  model = cd_fast.enet_coordinate_descent(
/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.076e+19, tolerance: 1.912e+16
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5, estimator=Pipeline(steps=[('ls', Lasso())]), n_jobs=-1,
             param_grid={'ls__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='r2')

In [18]:
y_pred = gs_ls.best_estimator_.predict(X_test)

print(f'Train rsquare: {gs_ls.score(X_train, y_train)}')
print(f'Test rsquare: {gs_ls.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.47603166883187364
Test rsquare: 0.37052452876916353
RMSE: 140439721.91866887


In [19]:
pipe_rf = Pipeline([('rf', RandomForestRegressor())]
                   )
params = {
            'rf__n_estimators': [50, 100, 200],
            'rf__max_features': ['auto', 'sqrt', 'log2']
         }

gs_rf = GridSearchCV(pipe_rf, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_rf.fit(X_train, y_train)

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(X

GridSearchCV(cv=5, estimator=Pipeline(steps=[('rf', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'rf__max_features': ['auto', 'sqrt', 'log2'],
                         'rf__n_estimators': [50, 100, 200]},
             scoring='r2')

In [20]:
y_pred = gs_rf.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_rf.score(X_train, y_train)}')
print(f'Test rsquare: {gs_rf.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.9165355641767452
Test rsquare: 0.4969204469330808
RMSE: 125550627.67523235


In [21]:
pipe_ada = Pipeline([('ada', AdaBoostRegressor())]
                   )
params = {
            'ada__n_estimators': [50, 100, 200],
            'ada__learning_rate': [0.01, 0.1, 0.2]
        }

gs_ada = GridSearchCV(pipe_ada, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_ada.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('ada', AdaBoostRegressor())]),
             n_jobs=-1,
             param_grid={'ada__learning_rate': [0.01, 0.1, 0.2],
                         'ada__n_estimators': [50, 100, 200]},
             scoring='r2')

In [22]:
y_pred = gs_ada.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_ada.score(X_train, y_train)}')
print(f'Test rsquare: {gs_ada.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.3721855479403572
Test rsquare: 0.34253037875761727
RMSE: 143528584.05414945


In [23]:
pipe_gbr = Pipeline([('gbr', GradientBoostingRegressor())]
                   )
params = {
            'gbr__n_estimators': [50, 100, 200],
            'gbr__learning_rate': [0.01, 0.1, 0.2],
            'gbr__max_depth': [3, 5, 7]
     
        }

gs_gbr = GridSearchCV(pipe_gbr, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_gbr.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('gbr', GradientBoostingRegressor())]),
             n_jobs=-1,
             param_grid={'gbr__learning_rate': [0.01, 0.1, 0.2],
                         'gbr__max_depth': [3, 5, 7],
                         'gbr__n_estimators': [50, 100, 200]},
             scoring='r2')

In [24]:
y_pred = gs_gbr.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_gbr.score(X_train, y_train)}')
print(f'Test rsquare: {gs_gbr.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred)}')

Train rsquare: 0.6459885502200007
Test rsquare: 0.4487450977039855
RMSE: 131424637.37071705
